# Day 10: Syntax Scoring

You ask the submarine to determine the best route out of the deep-sea cave, but it only replies:

Syntax error in navigation subsystem on line: all of them

All of them?! The damage is worse than you thought. You bring up a copy of the navigation subsystem (your puzzle input).

The navigation subsystem syntax is made of several lines containing chunks. There are one or more chunks on each line, and chunks contain zero or more other chunks. Adjacent chunks are not separated by any delimiter; if one chunk stops, the next chunk (if any) can immediately start. Every chunk must open and close with one of four legal pairs of matching characters:

    If a chunk opens with (, it must close with ).
    If a chunk opens with [, it must close with ].
    If a chunk opens with {, it must close with }.
    If a chunk opens with <, it must close with >.

So, () is a legal chunk that contains no other chunks, as is `[]`. More complex but valid chunks include `([])`, `{()()()}`, `<([{}])>`, `[<>({}){}[([])<>]]`, and even `(((((((((())))))))))`.

Some lines are incomplete, but others are corrupted. Find and discard the corrupted lines first.

A corrupted line is one where a chunk closes with the wrong character - that is, where the characters it opens and closes with do not form one of the four legal pairs listed above.

Examples of corrupted chunks include `(]`, `{()()()>`, `(((()))}`, and `<([]){()}[{}])`. Such a chunk can appear anywhere within a line, and its presence causes the whole line to be considered corrupted.

For example, consider the following navigation subsystem:

```text
[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]
```

Some of the lines aren't corrupted, just incomplete; you can ignore these lines for now. The remaining five lines are corrupted:

    {([(<{}[<>[]}>{[]{[(<()> - Expected ], but found } instead.
    [[<[([]))<([[{}[[()]]] - Expected ], but found ) instead.
    [{[{({}]{}}([{[{{{}}([] - Expected ), but found ] instead.
    [<(<(<(<{}))><([]([]() - Expected >, but found ) instead.
    <{([([[(<>()){}]>(<<{{ - Expected ], but found > instead.

Stop at the first incorrect closing character on each corrupted line.

Did you know that syntax checkers actually have contests to see who can get the high score for syntax errors in a file? It's true! To calculate the syntax error score for a line, take the first illegal character on the line and look it up in the following table:

    ): 3 points.
    ]: 57 points.
    }: 1197 points.
    >: 25137 points.

In the above example, an illegal `)` was found twice (2*3 = 6 points), an illegal `]` was found once (57 points), an illegal `}` was found once (1197 points), and an illegal `>` was found once (25137 points). So, the total syntax error score for this file is 6+57+1197+25137 = 26397 points!

Find the first illegal character in each corrupted line of the navigation subsystem. **What is the total syntax error score for those errors?**

In [1]:
# Python imports
from pathlib import Path
from typing import Callable, Dict, Generator, Iterable, List, Tuple

import numpy as np

# Paths to data
testpath = Path("day10_test.txt")
datapath = Path("day10_data.txt")

In [2]:
def load_input(fpath: Path) -> List[str]:
    """Read in lines from files as list of strings
    
    :param fpath:  Path to data file
    """
    with fpath.open("r") as ifh:
        return [_.strip() for _ in ifh.readlines()]

To check for syntax errors, we use the shunting algorithm (or something similar to it, anyway).

We step through the input string, character by character and, if it's a left bracket, we push it onto a stack. If it's a right bracket, we pop the stack and check that the brackets pair up. If they don't, it's a syntax error and we can handle it appropriately.

In [3]:
def has_syntax_error(line: str) -> str:
    """Returns first syntax error as character, or empty string
    
    :param line:  input line as string
    """
    line = list(line)  # convert line to string for stack operations
    lhs_stack = []  # holds seen characters (shunting algorithm)
    
    # While there are characters left to read, check next character.
    # If it's a right-hand bracket, pop the stack and check that
    # brackets match; if not, that's a syntax error so return the
    # character, otherwise continue.
    # If it's a left-hand bracket, push that onto the stack.
    while len(line):
        sym = line.pop(0)  # character to check
        if sym in ")]}>":  # character is right-hand bracket
            lhs = lhs_stack.pop()
            if f"{lhs}{sym}" not in ("()", "{}", "[]", "<>"): # syntax error
                return sym
        else:  # character is left-hand bracket
            lhs_stack.append(sym)

    return ""

def score_syntax_error(line: str) -> int:
    """Return syntax error score of passed line
    
    :param line:  input line as string

    Score is zero if no syntax error
    """
    scores = {")": 3, "]": 57, "}": 1197, ">": 25137, "": 0}
    
    return scores[has_syntax_error(line)]

With the test data:

In [4]:
score = sum([score_syntax_error(_) for _ in load_input(testpath)])
score

26397

And with the puzzle data:

In [5]:
score = sum([score_syntax_error(_) for _ in load_input(datapath)])
score

290691

## Puzzle 2:

Now, discard the corrupted lines. The remaining lines are incomplete.

Incomplete lines don't have any incorrect characters - instead, they're missing some closing characters at the end of the line. To repair the navigation subsystem, you just need to figure out the sequence of closing characters that complete all open chunks in the line.

You can only use closing characters `()`, `]`, `}`, or `>`), and you must add them in the correct order so that only legal pairs are formed and all chunks end up closed.

In the example above, there are five incomplete lines:

    [({(<(())[]>[[{[]{<()<>> - Complete by adding }}]])})].
    [(()[<>])]({[<{<<[]>>( - Complete by adding )}>]}).
    (((({<>}<{<{<>}{[]{[]{} - Complete by adding }}>}>)))).
    {<[[]]>}<{[{[{[]{()[[[] - Complete by adding ]]}}]}]}>.
    <{([{{}}[<[[[<>{}]]]>[]] - Complete by adding ])}>.

Did you know that autocomplete tools also have contests? It's true! The score is determined by considering the completion string character-by-character. Start with a total score of 0. Then, for each character, multiply the total score by 5 and then increase the total score by the point value given for the character in the following table:

    ): 1 point.
    ]: 2 points.
    }: 3 points.
    >: 4 points.

So, the last completion string above - `])}>` - would be scored as follows:

    Start with a total score of 0.
    Multiply the total score by 5 to get 0, then add the value of ] (2) to get a new total score of 2.
    Multiply the total score by 5 to get 10, then add the value of ) (1) to get a new total score of 11.
    Multiply the total score by 5 to get 55, then add the value of } (3) to get a new total score of 58.
    Multiply the total score by 5 to get 290, then add the value of > (4) to get a new total score of 294.

The five lines' completion strings have total scores as follows:

    }}]])})] - 288957 total points.
    )}>]}) - 5566 total points.
    }}>}>)))) - 1480781 total points.
    ]]}}]}]}> - 995444 total points.
    ])}> - 294 total points.

Autocomplete tools are an odd bunch: the winner is found by sorting all of the scores and then taking the middle score. (There will always be an odd number of scores to consider.) In this example, the middle score is 288957 because there are the same number of scores smaller and larger than it.

Find the completion string for each incomplete line, score the completion strings, and sort the scores. **What is the middle score?**

Finding incomplete lines is similar to finding syntax errors. We know the line is incomplete if, by the end of the input, we've found no errors, but there are still characters in the stack.

We can then work with the remaining stack to find the score.

In [6]:
def get_incomplete_lines(lines: List[str]) -> List[str]:
    """Return incomplete lines from passed set of lines
    
    :param lines:  list of input lines
    """
    return [_ for _ in lines if not(has_syntax_error(_))]

def score_incomplete_line(line: str) -> int:
    """Return score for an incomplete line
    
    :param line:  incomplete line as string
    """
    line = list(line)  # convert line to string for stack operations
    lhs_stack = []  # holds seen characters (shunting algorithm)
    
    # While there are characters left to read, check next character.
    # If it's a right-hand bracket, pop the stack and check that
    # brackets match; if not, that's a syntax error so return the
    # character, otherwise continue.
    # If it's a left-hand bracket, push that onto the stack.
    while len(line):
        sym = line.pop(0)  # character to check
        if sym in ")]}>":  # character is right-hand bracket
            lhs = lhs_stack.pop()
        else:  # character is left-hand bracket
            lhs_stack.append(sym)

    # Rather than complete the line, and then score the new completion
    # we step back through the stack and score the left-hand brackets
    scores = {"(": 1, "[": 2, "{": 3, "<": 4}
            
    score = 0
    while len(lhs_stack):
        score *= 5
        score += scores[lhs_stack.pop()]
    
    return score

With the test data:

In [7]:
scores = [score_incomplete_line(_) for _ in get_incomplete_lines(load_input(testpath))]
int(np.median(scores))

288957

With the input data:

In [8]:
scores = [score_incomplete_line(_) for _ in get_incomplete_lines(load_input(datapath))]
int(np.median(scores))

2768166558